In [3]:
import cv2
import os

# Path to the directory containing the images
train_folder_path = r'D:\JOY\J AMITY\Machine Learning\PInfotech\Task-3\data\train'

# Check if the directory exists
if not os.path.exists(train_folder_path):
    print("The specified folder does not exist.")
    exit()

# Read all images from the directory
images = []
for filename in os.listdir(train_folder_path):
            images.append(images)

# Now 'images' list contains all the images from the 'train' folder
print("Number of images found:", len(images))


Number of images found: 25000


In [4]:
import pandas as pd
import os

# Directory containing the .jpeg files
directory = r'D:\JOY\J AMITY\Machine Learning\PInfotech\Task-3\data\train'

# List all the files in the directory
filenames = os.listdir(directory)

# Initialize lists to store data
types = []
image_paths = []

# Iterate through filenames
for filename in filenames:
    # Construct the full path to the image
    image_path = os.path.join(directory, filename)
    # Split the filename and extension
    name, ext = os.path.splitext(filename)
    # Extract the type (first three characters)
    file_type = name[:3]
    # Append data to lists
    types.append(file_type)
    image_paths.append(image_path)

# Create DataFrame
df = pd.DataFrame({'type': types, 'image_path': image_paths})
df


,type,image_path
0,cat,D:\JOY\J AMITY\Machine Learning\PInfotech\Task...
1,cat,D:\JOY\J AMITY\Machine Learning\PInfotech\Task...
2,cat,D:\JOY\J AMITY\Machine Learning\PInfotech\Task...
3,cat,D:\JOY\J AMITY\Machine Learning\PInfotech\Task...
4,cat,D:\JOY\J AMITY\Machine Learning\PInfotech\Task...
...,...,...
24995,dog,D:\JOY\J AMITY\Machine Learning\PInfotech\Task...
24996,dog,D:\JOY\J AMITY\Machine Learning\PInfotech\Task...
24997,dog,D:\JOY\J AMITY\Machine Learning\PInfotech\Task...
24998,dog,D:\JOY\J AMITY\Machine Learning\PInfotech\Task...


In [5]:
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import svm
from tqdm import tqdm

# Define the preprocess_image function
def preprocess_image(image_path):
    img = imread(image_path)
    img = resize(img, (100, 100))
    img_flat = img.flatten()
    return img_flat

# Assuming you already have a DataFrame df containing 'image_path' and 'type' columns

X = []
y = df['type']
for image_path in tqdm(df['image_path'], desc="Preprocessing Images"):
    X.append(preprocess_image(image_path))
X = np.array(X)

# Perform PCA
pca = PCA(n_components=100)
X_pca = pca.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Train the SVM classifier
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)

Preprocessing Images: 100%|██████████████████████████████████████████████████████| 25000/25000 [11:52<00:00, 35.11it/s]


SVC()

In [6]:
from sklearn import metrics

y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = metrics.accuracy_score(y_test, y_pred)

# Print accuracy
print("Accuracy:", accuracy)


Accuracy: 0.6778


In [7]:
import cv2
import os

TEST_DIR = r'D:\JOY\J AMITY\Machine Learning\PInfotech\Task-3\data\test'

# Check if the directory exists
if not os.path.exists(TEST_DIR):
    print("The specified folder does not exist.")
    exit()

# Read all images from the directory
test_images = [os.path.join(TEST_DIR, i) for i in os.listdir(TEST_DIR)]

# Now 'test_images' list contains all the images from the 'test' folder with the prefix 'TEST_DIR'
print("Number of images found:", len(test_images))


Number of images found: 12500


In [9]:
def preprocess_image(test_images):
    img = imread(test_images)
    img = resize(img, (100, 100))
    img_flat = img.flatten()
    return img_flat

X = []
y = df['type']

# Use tqdm to track progress
for test_images in tqdm(df['image_path'], desc="Preprocessing Images"):
    X.append(preprocess_image(test_images))
X = np.array(X)

pca = PCA(n_components=100)
X_pca = pca.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

clf = svm.SVC(kernel='rbf')

clf.fit(X_train, y_train)

Preprocessing Images: 100%|██████████████████████████████████████████████████████| 25000/25000 [10:04<00:00, 41.36it/s]


SVC()

In [10]:
_pred = clf.predict(X_pca)

accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6778


In [13]:
test_predictions = clf.predict(X_test)

test_predictions_mapped = [1 if label == 'dog' else 0 for label in test_predictions]

predictions_df = pd.DataFrame({'id': range(1, len(test_predictions_mapped) + 1), 'label': test_predictions_mapped})

predictions_df.to_csv('svm_test_predictions.csv', index=False)

predictions_df

,id,label
0,1,0
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
4995,4996,1
4996,4997,0
4997,4998,0
4998,4999,0
